## Language Modeling
* dataset= https://sourceforge.net/projects/kalimat/files/kalimat/document-collection/

In [1]:
import os

In [2]:
import util as ut


Bi-grams

In [3]:

test_text1 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون والاذاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"
err_text1 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"

test_text2 = "حقق منتخبنا الوطني الاول لكرة القدم كل اهدافه المطلوبة"
err_text2 = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
err_text2 = "منتخبنا يتألق ويهم الشباك"


In [4]:
# using bigger corpus
def CheckNGramInCorpus(texttoken,corpustoken): #return ngram that don't exist in corpustoken
    ngram = [g for g in texttoken if g not in corpustoken]
    return (ngram)

def WrongNGram(text):
    token = ut.get_all_ngrams(text, nrange=2)
    ngram_i = []
    for directory in ["articlesCulture","articlesInternational","articlesSports"]:
        for filename in os.listdir("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/"+directory):
            f = open("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/" +
                     directory+"/"+filename, "r", encoding="utf8")
            ngram_i = ut.get_all_ngrams(f.read().replace('\n',' '), nrange=2) 
            #print(token)
            token = CheckNGramInCorpus(token,ngram_i)
            if len(token)==0:
                print(token ,'no error')
                return(token)
            f.close()        
    print("potential error in : \n ",token)
    return(token)
                    


# improvements 1: 
add cleaning, lemmatizing + removing stop words

In [6]:
#lemmatization
#! pip install qalsadi
import qalsadi.lemmatizer
lemmer = qalsadi.lemmatizer.Lemmatizer()
def lemmatize(text):
    t=lemmer.lemmatize_text(text)
    s = ut.stringify(t)
    return s


In [7]:
#Stemming (meth non utilisée)
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()

def stemmatize(text):
    tokens = text.split()
    stem_tokens = [stemmer.suf32(token) for token in tokens]
    s= ut.stringify(stem_tokens)
    return(s)

In [142]:
def CheckNGramInCorpus(texttoken,corpustoken): #return ngram that don't exist in corpustoken
    ngram = [g for g in texttoken if g not in corpustoken]
    return (ngram)
def WrongNGramV2(text,n = 2):
    text = ut.very_clean(text)
    text = lemmatize(text)
    token = ut.get_all_ngrams(text,nrange = n)
    ngram_i = []
    for directory in ["articlesCulture", "articlesInternational", "articlesSports"]:
        for filename in os.listdir("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/"+directory):
            f = open("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/" +
                     directory+"/"+filename, "r", encoding="utf8")
            g = ut.very_clean(f.read().replace('\n', ' '))
            g = lemmatize(g)
            ngram_i = ut.get_all_ngrams(g,nrange=n)
            #print(token)
            token = CheckNGramInCorpus(token,ngram_i)
            if len(token)==0:
                #print(token ,'no error')
                return(token)
            f.close()        
    #print("potential error in : \n ",token)
    return(token)

## Improvements 2 : 
(la meth precedente est tres lourde : parcourir tous le fichier pour prouver que le ngram n'existe pas)
--> solution : sortir de la boucle si on retrouve un ngram tres proche (inversion/suppression/ajout d'un caratere) :

In [1]:
# return ngram that don't exist in corpustoken or exist with minor change (typography mistake)
def OneSubstitution(word, word2):# return true if exists one and only onesubstituion error
    if len(word)==len(word2):
        l = []
        for i in range(len(word)):
            if word[i] == word2[i]:
                l.append(0)
            else:
                l.append(1)
        return (sum(l) == 1)
    else:
        return(False)


def OneInversion(word, word2):
    if len(word) == len(word2):  # returns true if exists one and only inversion error
        l1 = []
        l2 = []
        for i in range(len(word)):
            if word[i] == word2[i]:
                l1.append(0)
            else:
                l1.append(1)
                l2.append(word[i])
                l2.append(word2[i])
        if sum(l1) == 2:
          return(l2[0] == l2[3] and l2[1] == l2[2])
        else:
            return(False)
    else:
        return(False)


def AddSubErr(word1, word2):  # ajout/suppression d'un seul caracter
    w1 = set(word1)
    w2 = set(word2)
    maxi=max(len(w1-w2),len(w2-w1))
    if maxi != 1:
        return(False)
    else :
        if len(w1)-len(w2) == 1:
            lw1=list(word1)
            lw1.remove(''.join(w1-w2))
            s = ''.join(str(x) for x in lw1)
            return(s==word2)
        elif len(w1)-len(w2) == -1:
            lw2 = list(word2)
            lw2.remove(''.join(w2-w1))
            s = ''.join(str(x) for x in lw2)
            return(s == word1)
        else:
            return(False)
def lookslike(words, corpustoken):
    for word in words:
        for c in corpustoken:
            if AddSubErr(word,c): 
                return(word)
            elif OneInversion(word,c) or OneSubstitution(word,c) : #inversion / substitution
                return(word)
    return('')

def CheckNGramInCorpus(texttoken, corpustoken):
    ngram = [g for g in texttoken if g not in corpustoken]
    return (ngram)


In [2]:
def WrongNGramV3(text, n=2):
    text = ut.very_clean(text)
    #text = lemmatize(text)
    token = ut.get_all_ngrams(text, nrange=n)
    ngram_i = []
    err = ''
    for directory in ["articlesSports","articlesCulture", "articlesInternational"]:
        for filename in os.listdir("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/"+directory):
            f = open("C:/Users/PC/Desktop/ALL/II3IA/NLP/articles/" +
                     directory+"/"+filename, "r", encoding="utf8")
            g = ut.very_clean(f.read().replace('\n', ' '))
            #g = lemmatize(g)
            ngram_i = ut.get_all_ngrams(g, nrange=n)
            #print(token)
            #print(ngram_i)
            token = CheckNGramInCorpus(token, ngram_i)
            err = lookslike(token,ngram_i)
            if len(err) !=0 :
                print("potential error in: ",err) 
            if len(token) == 0:
                print(token ,'no error')
                return(token)
            f.close()
    print("potential error in :", err)
    print("error in : \n ",token)
    return(token)


## Execute

In [3]:
def mispelled_word (text,n=2): #sans considerer lemmatizationb
    #l = WrongNGram(text)
    #l = WrongNGramV2(text)
    l = WrongNGramV3(text)
    print(l)
    if len(l)>1:
        l2 = l[0].split("_")
        print("mispelled word is : ")
        return(l2[-1])
    elif len(l) == 1 : #mistake in first or last word
        print("the error is either first or last word")
        return(l)
    else :
        print("no error in this sentence")
        return(None)


In [4]:
WrongNGramV3(err_text2)

NameError: name 'err_text2' is not defined

## combinaison : modele de language avec model AraVec

n-grams avec model AraVec 

In [ ]:
import gensim


In [ ]:

model = gensim.models.Word2Vec.load(
    "C:/Users/PC/Desktop/ALL/II3IA/NLP/full_grams_cbow_100_twitter.mdl")
model.wv.save_word2vec_format("./spacyModel/aravec.txt")


In [ ]:

text1 = "لاتعطني سمكا بل علمني كيف اصطاد"
ut.get_all_ngrams(text1, nrange=4)

print("n-grams existing in vocabulary")
for token in ut.get_all_ngrams(text1, nrange=4):
    if token in model.wv:
        print(token)
        # only one bigram exists
print("words existing in vocabulary")
for token in text1.split():
    if token in model.wv:
        print(token)
        # only one word exists
